In [1]:
import pandas as pd
import subprocess

In [2]:
!mkdir "/n/netscratch/extavour_lab/Everyone/Rishabh/TSA_copepod"

In [2]:
df=pd.read_csv("Copepod_tsas.csv",index_col=0)
df_map={x:y for x,y in zip(df.index,df.Species)}

In [11]:
##download tsas using wget from SRA 
for prefix in df.index:
    try:
        ##extract prefixes for indexing into sra traces 
        n1=prefix[0:2]
        n2=prefix[2:4]
        ##obtain the wgs number
        a=subprocess.check_output(["curl","-s",f"https://locate.ncbi.nlm.nih.gov/sdl/2/retrieve?acc={prefix}000000"])
        wgs_number=str(a).split('"link":')[1].split("traces/")[1].split("/WGS/")[0]
        address=f"https://sra-download.ncbi.nlm.nih.gov/traces/{wgs_number}/wgs_aux/{n1}/{n2}/{prefix}/{prefix}.1.fsa_nt.gz"
        a=subprocess.check_output(["wget",address])
        subprocess.run(f"gunzip -c {prefix}.1.fsa_nt.gz > /n/netscratch/extavour_lab/Everyone/Rishabh/TSA_copepod/{prefix}.1.fasta", shell=True)
        !rm "$prefix".1.fsa_nt.gz
    except:
        print(prefix)

--2025-08-31 20:45:56--  https://sra-download.ncbi.nlm.nih.gov/traces/wgs03/wgs_aux/HA/HV/HAHV01/HAHV01.1.fsa_nt.gz
Resolving sra-download.ncbi.nlm.nih.gov... 130.14.250.25, 130.14.250.27, 2607:f220:41e:250::27, ...
Connecting to sra-download.ncbi.nlm.nih.gov|130.14.250.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18265167 (17M) [application/octet-stream]
Saving to: ‘HAHV01.1.fsa_nt.gz’

     0K .......... .......... .......... .......... ..........  0% 1.90M 9s
    50K .......... .......... .......... .......... ..........  0% 3.36M 7s
   100K .......... .......... .......... .......... ..........  0%  125M 5s
   150K .......... .......... .......... .......... ..........  1% 3.45M 5s
   200K .......... .......... .......... .......... ..........  1%  127M 4s
   250K .......... .......... .......... .......... ..........  1%  122M 3s
   300K .......... .......... .......... .......... ..........  1% 88.9M 3s
   350K .......... .......... .......... ....

In [18]:
## run transdecoder on each tsa
import multiprocessing as mp
def trans(x):
    !sh copepod_transdecoder.sh "$x"
with mp.Pool(6) as p:
    p.map(trans,list(df.index))

CMD: mkdir -p /n/netscratch/extavour_lab/Everyone/Rishabh/TSA_copepod/TSA_transdec/GJAO01/GJAO01.1.fasta.transdecoder_dir
* [Mon Sep  1 00:54:42 2025] Running CMD: /usr/local/opt/transdecoder/util/compute_base_probs.pl GJAO01.1.fasta 0 > /n/netscratch/extavour_lab/Everyone/Rishabh/TSA_copepod/TSA_transdec/GJAO01/GJAO01.1.fasta.transdecoder_dir/base_freqs.dat


-first extracting base frequencies, we'll need them later.
* [Mon Sep  1 00:54:42 2025] Running CMD: /usr/local/opt/transdecoder/util/compute_base_probs.pl GCHA01.1.fasta 0 > /n/netscratch/extavour_lab/Everyone/Rishabh/TSA_copepod/TSA_transdec/GCHA01/GCHA01.1.fasta.transdecoder_dir/base_freqs.dat


-first extracting base frequencies, we'll need them later.
CMD: mkdir -p /n/netscratch/extavour_lab/Everyone/Rishabh/TSA_copepod/TSA_transdec/GJRL01/GJRL01.1.fasta.transdecoder_dir
* [Mon Sep  1 00:54:42 2025] Running CMD: /usr/local/opt/transdecoder/util/compute_base_probs.pl GJRL01.1.fasta 0 > /n/netscratch/extavour_lab/Everyone/Rish

In [1]:
## combine translations into a single fasta
%%bash
cd /n/netscratch/extavour_lab/Everyone/Rishabh/TSA_copepod/TSA_transdec
cat */*.pep > combined_protein.fasta

In [2]:
!mkdir "tsa_cluster_14"

In [3]:
## run hmmsearch of the hgt interval against the protein db
%%bash

hgt_hmm="outputs/hmmbuild/GCF_007210705.1;XP_059092480.1/GCF_007210705.1;XP_059092480.1;HGT_(88,215)/sub_seq.hmm"
prot="/n/netscratch/extavour_lab/Everyone/Rishabh/TSA_copepod/TSA_transdec/combined_protein.fasta"
singularity exec /cvmfs/singularity.galaxyproject.org/h/m/hmmer:3.3.2--he1b5a44_0 hmmsearch -E 1e-2 --cpu 2 --noali --domtblout "tsa_cluster_14/hgt.domtblout" "$hgt_hmm"  "$prot"

# hmmsearch :: search profile(s) against a sequence database
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query HMM file:                  outputs/hmmbuild/GCF_007210705.1;XP_059092480.1/GCF_007210705.1;XP_059092480.1;HGT_(88,215)/sub_seq.hmm
# target sequence database:        /n/netscratch/extavour_lab/Everyone/Rishabh/TSA_copepod/TSA_transdec/combined_protein.fasta
# per-dom hits tabular output:     tsa_cluster_14/hgt.domtblout
# show alignments in output:       no
# sequence reporting threshold:    E-value <= 0.01
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       GCF_007210705.1;XP_059092480.1;HGT_(88,215)  [M=143]
Scores for complete sequences (score includes all domains):
   --- full sequence ---   --- best 1 domain ---    -#dom-
    E-value  score  bias    E-va

In [4]:
## run hmmsearch of the meta interval against the protein db
%%bash

hgt_hmm="outputs/hmmbuild/GCF_007210705.1;XP_059092480.1/GCF_007210705.1;XP_059092480.1;Meta_(26,83)/sub_seq.hmm"
prot="/n/netscratch/extavour_lab/Everyone/Rishabh/TSA_copepod/TSA_transdec/combined_protein.fasta"
singularity exec /cvmfs/singularity.galaxyproject.org/h/m/hmmer:3.3.2--he1b5a44_0 hmmsearch -E 1e-2 --cpu 2 --noali --domtblout "tsa_cluster_14/meta.domtblout" "$hgt_hmm"  "$prot"

# hmmsearch :: search profile(s) against a sequence database
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query HMM file:                  outputs/hmmbuild/GCF_007210705.1;XP_059092480.1/GCF_007210705.1;XP_059092480.1;Meta_(26,83)/sub_seq.hmm
# target sequence database:        /n/netscratch/extavour_lab/Everyone/Rishabh/TSA_copepod/TSA_transdec/combined_protein.fasta
# per-dom hits tabular output:     tsa_cluster_14/meta.domtblout
# show alignments in output:       no
# sequence reporting threshold:    E-value <= 0.01
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       GCF_007210705.1;XP_059092480.1;Meta_(26,83)  [M=68]
Scores for complete sequences (score includes all domains):
   --- full sequence ---   --- best 1 domain ---    -#dom-
    E-value  score  bias    E-va

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [5]:
##process hmmsearch outputs
!sh split_hmmer.sh "tsa_cluster_14/hgt.domtblout"  "tsa_cluster_14/hgt.domtblout.tsv"
!sh split_hmmer.sh "tsa_cluster_14/meta.domtblout"  "tsa_cluster_14/meta.domtblout.tsv"

In [99]:
dfhgt=pd.read_csv("tsa_cluster_14/hgt.domtblout.tsv",sep="\t",names=["target_name", "target_accession","tlen", "query name","accession","qlen", "E-value", "score1", "bias1",   "#", "of", "c-Evalue",  "i-Evalue",  "score",  "bias",  "hmmfrom",  "hmmto",    "alifrom",  "alito", "envfrom",  "envto", "acc", "description of target", "species"])
for index, row in dfhgt.iterrows():
    dfhgt.loc[index,['species']]=df_map[row.target_name[0:6]]
dfmeta=pd.read_csv("tsa_cluster_14/meta.domtblout.tsv",sep="\t",names=["target_name", "target_accession","tlen", "query name","accession","qlen", "E-value", "score1", "bias1",   "#", "of", "c-Evalue",  "i-Evalue",  "score",  "bias",  "hmmfrom",  "hmmto",    "alifrom",  "alito", "envfrom",  "envto", "acc", "description of target", "species"])
for index, row in dfmeta.iterrows():
    dfmeta.loc[index,['species']]=df_map[row.target_name[0:6]]

/tmp/ipykernel_1335869/2516388114.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Tigriopus japonicus' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dfhgt.loc[index,['species']]=df_map[row.target_name[0:6]]
/tmp/ipykernel_1335869/2516388114.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Metridia pacifica' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dfmeta.loc[index,['species']]=df_map[row.target_name[0:6]]


In [101]:
## identify hgt-chimeras as double positive hits for hgt and metazoan domains
chimeras_hgt=dfhgt[dfhgt.target_name.isin(set(dfhgt.target_name)&set(dfmeta.target_name))]
chimeras_hgt['cov']=[y-x for x,y in zip(chimeras_hgt.envfrom, chimeras_hgt.envto)]
l=ast.literal_eval(chimeras_hgt['query name'].values[0].split("_")[-1])
l=l[1]-l[0]
chimeras_hgt=chimeras_hgt[chimeras_hgt['cov']>.30*l]
chimeras_hgt=chimeras_hgt[chimeras_hgt.target_name.isin(set(chimeras_hgt.target_name)&set(chimeras_meta.target_name))]

chimeras_hgt=chimeras_hgt.loc[chimeras_hgt.groupby("species")["i-Evalue"].idxmin()]

chimeras_meta=dfmeta.copy()
chimeras_meta['cov']=[y-x for x,y in zip(chimeras_meta.envfrom, chimeras_meta.envto)]
l=ast.literal_eval(chimeras_meta['query name'].values[0].split("_")[-1])
l=l[1]-l[0]
chimeras_meta=chimeras_meta[chimeras_meta['cov']>.30*l]
chimeras_meta=chimeras_meta[chimeras_meta.target_name.isin(set(chimeras_hgt.target_name)&set(chimeras_meta.target_name))]


/tmp/ipykernel_1335869/2604857101.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chimeras_hgt['cov']=[y-x for x,y in zip(chimeras_hgt.envfrom, chimeras_hgt.envto)]


In [104]:
##verify appropriate order of hgt and meta intervals
for index in set(chimeras_hgt.target_name):
    print(chimeras_meta[chimeras_meta.target_name==index].loc[:,["envfrom","envto"]].values[0],chimeras_hgt[chimeras_hgt.target_name==index].loc[:,["envfrom","envto"]].values[0])

[18 85] [ 81 225]
[ 7 77] [226 361]
[ 8 75] [ 71 214]
[ 6 75] [ 71 214]
[ 8 75] [ 71 214]
[ 1 29] [ 25 166]


In [105]:
from Bio import SeqIO

# Load FASTA file into a dictionary
fasta_dict = SeqIO.to_dict(SeqIO.parse("/n/netscratch/extavour_lab/Everyone/Rishabh/TSA_copepod/TSA_transdec/combined_protein.fasta", "fasta"))

In [118]:
## write hgt intervals to a fasta
f=open('tsa_cluster_14/new_hgt_chimera_seqs.fasta','w')
for index in set(chimeras_hgt.target_name):
    start,stop=chimeras_hgt[chimeras_hgt.target_name==index].loc[:,["envfrom","envto"]].values[0]
    s=str(fasta_dict[index].seq)[start-1:stop]
    f.write(f">{index}\n")
    f.write(f"{s}\n")
f.close()

In [119]:
## write meta intervals to a fasta
f=open('tsa_cluster_14/new_meta_chimera_seqs.fasta','w')
for index in set(chimeras_hgt.target_name):
    start,stop=chimeras_meta[chimeras_meta.target_name==index].loc[:,["envfrom","envto"]].values[0]
    s=str(fasta_dict[index].seq)[start-1:stop]
    f.write(f">{index}\n")
    f.write(f"{s}\n")
f.close()

In [127]:
!cp "outputs/phylogenetic_data_filtered/GCF_007210705.1;XP_059092480.1;HGT_(88,215)/all_sequences.fa" tsa_cluster_14/og_hgt.fasta

In [128]:
!cp "outputs/phylogenetic_data_filtered/GCF_007210705.1;XP_059092480.1;Meta_(26,83)/all_sequences.fa" tsa_cluster_14/og_meta.fasta

In [129]:
!cat 'tsa_cluster_14/og_hgt.fasta' 'tsa_cluster_14/new_hgt_chimera_seqs.fasta' > 'tsa_cluster_14/add_new_hgt_chimera_seqs.fasta'

In [132]:
!cat 'tsa_cluster_14/og_meta.fasta' 'tsa_cluster_14/new_meta_chimera_seqs.fasta' > 'tsa_cluster_14/add_new_meta_chimera_seqs.fasta'

In [149]:
## run iqtree 
!sbatch run_iqtree_cluster14_tsas.sh add_new_hgt_chimera_seqs

Submitted batch job 33027563


In [150]:
!sbatch run_iqtree_cluster14_tsas.sh add_new_meta_chimera_seqs

Submitted batch job 33027565


In [ ]:
meta_only.to_csv('tsa_cluster_14/new_meta_only.tsv',sep='\t')

In [148]:
hgt_only.to_csv('tsa_cluster_14/new_hgt_only.tsv',sep='\t')

In [3]:
import os
from ete3 import Tree

/n/home11/rkapoor/.conda/envs/rishabh/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [68]:
## edit tree labels
for x in os.listdir("tsa_cluster_14"):
    if '.treefile' in x and 'edited_' not in x:
        T=Tree(f"tsa_cluster_14/{x}")
        for xt in T:
            xt.name=xt.name.split("__")[0]
            if "__" in xt.name:
                xt.name="_".join(xt.name.split("_")[2:])
            xt.name=xt.name.split("_HGT")[0].split("_Meta")[0]
            if len(xt.name.split("_"))>2:
                xt.name="_".join(xt.name.split("_")[2:])
        T.write(format=0, outfile=f"tsa_cluster_14/edited_{x}")
dfc=pd.read_csv("Copepod_tsas.csv",index_col=0)

In [56]:
for x in os.listdir("tsa_cluster_14"):
    if '.tsv' in x:
        df=pd.read_csv(f"tsa_cluster_14/{x}",sep="\t",index_col=0)
        df=df[df.target_name.astype(str)!='nan']
        for index, row in df.iterrows():
            df.loc[index,['species','order','class','phylum']]=dfc.loc[row['target_name'][0:6],['Species','Order','Class','Phylum',]].values
            df.to_csv(f"tsa_cluster_14/{x}",sep="\t")

/tmp/ipykernel_1738869/3939680783.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Tigriopus japonicus' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[index,['species','order','class','phylum']]=dfc.loc[row['target_name'][0:6],['Species','Order','Class','Phylum',]].values
/tmp/ipykernel_1738869/3939680783.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Metridia pacifica' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[index,['species','order','class','phylum']]=dfc.loc[row['target_name'][0:6],['Species','Order','Class','Phylum',]].values


In [61]:
##write itol taxonomic info file
f=open("tsa_cluster_14/HGT_itol_taxonomic_info.txt",'a')
r=",-1,#000000,normal,1,0"
df=pd.read_csv("tsa_cluster_14/new_hgt_only.tsv",index_col=0,sep="\t")
for index, row in df.iterrows():
    s=row['target_name']+","+";".join(row[['species','order','class','phylum']].values)+r+"\n"
    f.write(s)
df=pd.read_csv("tsa_cluster_14/new_hgt_chimeras.tsv",index_col=0,sep="\t")
for index, row in df.iterrows():
    s=row['target_name']+","+";".join(row[['species','order','class','phylum']].values)+r+"\n"
    f.write(s)
f.close()

f=open("tsa_cluster_14/meta_itol_taxonomic_info.txt",'a')
r=",-1,#000000,normal,1,0"
df=pd.read_csv("tsa_cluster_14/new_meta_only.tsv",index_col=0,sep="\t")
for index, row in df.iterrows():
    s=row['target_name']+","+";".join(row[['species','order','class','phylum']].values)+r+"\n"
    f.write(s)
df=pd.read_csv("tsa_cluster_14/new_meta_chimeras.tsv",index_col=0,sep="\t")
for index, row in df.iterrows():
    s=row['target_name']+","+";".join(row[['species','order','class','phylum']].values)+r+"\n"
    f.write(s)
f.close()

In [63]:
##write itol color strip
f=open("tsa_cluster_14/HGT_itol_color_strip.txt",'a')
r=",-1,#000000,normal,1,0"
df=pd.read_csv("tsa_cluster_14/new_hgt_only.tsv",index_col=0,sep="\t")
for index, row in df.iterrows():
    s=row['target_name']+" "+"#ffc0cb\n"
    f.write(s)
df=pd.read_csv("tsa_cluster_14/new_hgt_chimeras.tsv",index_col=0,sep="\t")
for index, row in df.iterrows():
    s=row['target_name']+" "+"#ff0000\n"
    f.write(s)
f.close()

f=open("tsa_cluster_14/meta_itol_color_strip.txt",'a')
r=",-1,#000000,normal,1,0"
df=pd.read_csv("tsa_cluster_14/new_meta_only.tsv",index_col=0,sep="\t")
for index, row in df.iterrows():
    s=row['target_name']+" "+"#ffc0cb\n"
    f.write(s)
df=pd.read_csv("tsa_cluster_14/new_meta_chimeras.tsv",index_col=0,sep="\t")
for index, row in df.iterrows():
    s=row['target_name']+" "+"#ff0000\n"
    f.write(s)
f.close()